In [ ]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from collections import Counter
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import os

# --- Spotify App Credentials ---


In [ ]:
CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID"),
CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET"),
REDIRECT_URI = os.getenv("SPOTIPY_REDIRECT_URI"),


# --- Scopes Required for Audio Features ---


In [4]:
SCOPE = "user-read-private user-top-read user-read-recently-played"

# --- Authenticate ---

In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

# --- Example Track IDs ---

In [6]:
track_ids = [
    "11dFghVXANMlKmJXsNCbNl",  # Mr. Brightside
    "3yqF4F1Kk10VdUuIKOXzx6",  # Another track
    # Add more track IDs here
]

# --- Fetch Audio Features ---

In [7]:
features = sp.audio_features(track_ids)

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=11dFghVXANMlKmJXsNCbNl,3yqF4F1Kk10VdUuIKOXzx6 with Params: {} returned 403 due to None


SpotifyException: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=11dFghVXANMlKmJXsNCbNl,3yqF4F1Kk10VdUuIKOXzx6:
 None, reason: None

# --- Convert to DataFrame ---

In [8]:
df = pd.DataFrame(features)
df = df.dropna(subset=["id"])  # Remove any failed lookups

NameError: name 'features' is not defined

# --- Display ---

In [ ]:
df[["id", "danceability", "energy", "tempo", "valence", "acousticness", "instrumentalness"]]